In [15]:
import psycopg2

In [16]:
# Подключение к PostgreSQL без указания базы данных
conn = psycopg2.connect(
    host="localhost",
    database="postgres",  # Подключаемся к стандартной базе данных
    user="postgres",
    password="1"
)

conn.autocommit = True  # Включаем режим автокоммита для выполнения CREATE DATABASE
cursor = conn.cursor()

In [17]:
# Create new database
cursor.execute("CREATE DATABASE knowledge;")
cursor.close()
conn.close()

DuplicateDatabase: database "knowledge" already exists


In [18]:
# Функция для подключения к базе данных
def connect_db():
    return psycopg2.connect(
        dbname="knowledge",  # Your database name
        user="postgres",  # Your username
        password="1",  # Your password
        host="localhost"  # Host, use "localhost" if running locally
    )

In [19]:
# Функция для создания таблиц в базе данных
def create_database_structure():
    conn = connect_db()
    cursor = conn.cursor()

    # Создаем таблицы
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS knowledge_groups (
            id SERIAL PRIMARY KEY,
            category_name VARCHAR(255) UNIQUE NOT NULL
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS knowledge_items (
            id SERIAL PRIMARY KEY,
            item_name VARCHAR(255) UNIQUE NOT NULL,
            group_id INTEGER REFERENCES knowledge_groups(id) ON DELETE CASCADE,
            is_known BOOLEAN DEFAULT FALSE
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS learning_units (
            id SERIAL PRIMARY KEY,
            main_knowledge INTEGER REFERENCES knowledge_items(id) ON DELETE CASCADE,
            text VARCHAR(500) NOT NULL,
            group_name VARCHAR(255) NOT NULL
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS unit_knowledges (
            id SERIAL PRIMARY KEY,
            unit_id INTEGER REFERENCES learning_units(id) ON DELETE CASCADE,
            knowledge_id INTEGER REFERENCES knowledge_items(id) ON DELETE CASCADE
        );
    """)

    conn.commit()
    cursor.close()
    conn.close()

In [13]:
def check_database_structure():
    conn = connect_db()
    cursor = conn.cursor()

    # Check for the presence of tables
    cursor.execute("""
        SELECT table_name FROM information_schema.tables
        WHERE table_schema = 'public';
    """)
    tables = cursor.fetchall()
    print("\nTables in the database:")
    for table in tables:
        print(f"- {table[0]}")

    # Check the structure of each table
    for table in tables:
        print(f"\nStructure of table {table[0]}:")
        cursor.execute(f"""
            SELECT column_name, data_type, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_name = '{table[0]}';
        """)
        columns = cursor.fetchall()
        for column in columns:
            print(f"  Column: {column[0]}")
            print(f"    Data type: {column[1]}")
            print(f"    Nullable: {column[2]}")
            print(f"    Default value: {column[3]}")
    
    conn.close()

In [20]:
# Run the database structure check function
if __name__ == "__main__":
    check_database_structure()


Tables in the database:
